# 04 - Spark DataFrame Operations

Nauka podstawowych operacji na DataFrame w Apache Spark na danych MovieLens.

**Tematy:**
- Tworzenie SparkSession i ładowanie danych
- Inspekcja schematu (printSchema, dtypes, describe)
- select, selectExpr
- filter / where
- withColumn, drop, withColumnRenamed
- sort / orderBy
- join (inner, left, right, cross)
- union, distinct, dropDuplicates
- limit, sample, take

## 1. SparkSession + ładowanie danych z PostgreSQL

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("04_DataFrame_Operations") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

In [ ]:
ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

print(f"Ratings: {ratings.count()} rows")
print(f"Movies: {movies.count()} rows")

## 2. Inspekcja schematu

Zanim zaczniesz pracę z DataFrame, zawsze warto sprawdzić jego strukturę.

In [ ]:
# printSchema() - pełna struktura z typami danych
ratings.printSchema()

In [ ]:
# dtypes - lista krotek (nazwa, typ)
print(ratings.dtypes)

# columns - lista nazw kolumn
print(ratings.columns)

In [ ]:
# describe() - statystyki dla kolumn numerycznych
ratings.describe().show()

In [ ]:
# summary() - rozszerzone statystyki (25%, 50%, 75% percentyle)
ratings.summary().show()

## 3. select i selectExpr

`select` - wybiera kolumny (jak SELECT w SQL)  
`selectExpr` - pozwala używać wyrażeń SQL jako stringów

In [ ]:
from pyspark.sql.functions import col, lit, upper, lower, length

# select - po nazwie kolumny
ratings.select("user_id", "movie_id", "rating").show(5)

# select - z użyciem col()
ratings.select(col("user_id"), col("rating") * 2).show(5)

In [ ]:
# selectExpr - wyrażenia SQL jako stringi
ratings.selectExpr(
    "user_id",
    "movie_id",
    "rating",
    "rating * 2 as double_rating",
    "CASE WHEN rating >= 4.0 THEN 'high' ELSE 'low' END as rating_category"
).show(10)

### Zadanie 1
Wybierz z tabeli `movies` kolumny `title` i `genres`, dodaj kolumnę `title_length` z długością tytułu.

In [ ]:
# Twoje rozwiązanie:


## 4. filter / where

Filtrowanie wierszy - `filter` i `where` działają identycznie.

In [ ]:
# Filtrowanie po wartości
high_ratings = ratings.filter(col("rating") >= 4.5)
print(f"Oceny >= 4.5: {high_ratings.count()}")
high_ratings.show(5)

In [ ]:
# Filtrowanie z użyciem stringa SQL
ratings.where("rating >= 4.5 AND user_id < 100").show(5)

In [ ]:
# Łączenie warunków: & (AND), | (OR), ~ (NOT)
ratings.filter(
    (col("rating") >= 4.0) & (col("user_id").between(1, 50))
).show(5)

# isNull / isNotNull
movies.filter(col("genres").isNotNull()).count()

In [ ]:
# isin - filtrowanie po liście wartości
selected_users = [1, 42, 100, 500]
ratings.filter(col("user_id").isin(selected_users)).show(10)

In [ ]:
# like / rlike (regex) na stringach
movies.filter(col("title").like("%Toy Story%")).show()

# Filmy z roku 2015 (regex)
movies.filter(col("title").rlike(r"\(2015\)")).show(5)

### Zadanie 2
Znajdź wszystkie filmy, które mają gatunek "Comedy" I zostały wydane po roku 2000 (użyj rlike na tytule).

In [ ]:
# Twoje rozwiązanie:


## 5. withColumn, drop, withColumnRenamed

Modyfikacja kolumn - Spark DataFrame jest niemutowalny, więc każda operacja zwraca nowy DataFrame.

In [ ]:
from pyspark.sql.functions import when, regexp_extract, split, size, year, to_timestamp

# withColumn - dodaj nową kolumnę lub nadpisz istniejącą
movies_enriched = movies \
    .withColumn("year", regexp_extract(col("title"), r"\((\d{4})\)", 1).cast("int")) \
    .withColumn("genre_count", size(split(col("genres"), "\\|"))) \
    .withColumn("is_comedy", col("genres").contains("Comedy"))

movies_enriched.show(10)

In [ ]:
# withColumnRenamed
movies_renamed = movies.withColumnRenamed("movie_id", "id") \
                       .withColumnRenamed("title", "movie_title")
movies_renamed.printSchema()

In [ ]:
# drop - usuń kolumnę
ratings_slim = ratings.drop("rating_timestamp")
ratings_slim.printSchema()

In [ ]:
# when / otherwise - odpowiednik CASE WHEN
ratings_labeled = ratings.withColumn(
    "rating_label",
    when(col("rating") >= 4.0, "positive")
    .when(col("rating") >= 3.0, "neutral")
    .otherwise("negative")
)
ratings_labeled.show(10)

### Zadanie 3
Dodaj do `ratings` kolumnę `decade` - dekadę, w której film został oceniony (na podstawie `rating_timestamp`).
Np. 2005 → 2000, 2013 → 2010.

In [ ]:
# Twoje rozwiązanie:


## 6. sort / orderBy

In [ ]:
from pyspark.sql.functions import desc, asc

# Sortowanie rosnąco (domyślnie)
movies.orderBy("title").show(5)

# Sortowanie malejąco
movies.orderBy(desc("movie_id")).show(5)

# Sortowanie po wielu kolumnach
ratings.orderBy(asc("user_id"), desc("rating")).show(10)

## 7. join

Łączenie DataFrames - kluczowa operacja w Spark.

Typy joinów:
- `inner` (domyślny) - tylko dopasowane wiersze
- `left` / `left_outer` - wszystko z lewej + dopasowane z prawej
- `right` / `right_outer` - wszystko z prawej + dopasowane z lewej
- `full` / `full_outer` / `outer` - wszystko z obu stron
- `cross` - iloczyn kartezjański
- `left_semi` - wiersze z lewej, które mają dopasowanie (bez kolumn z prawej)
- `left_anti` - wiersze z lewej, które NIE mają dopasowania

In [ ]:
# inner join - oceny z tytułami filmów
ratings_with_titles = ratings.join(movies, "movie_id")
ratings_with_titles.show(5)

In [ ]:
# left_anti - filmy BEZ żadnej oceny
movies_without_ratings = movies.join(ratings, "movie_id", "left_anti")
print(f"Filmy bez ocen: {movies_without_ratings.count()}")
movies_without_ratings.show(5)

In [ ]:
# left_semi - filmy które MAJĄ przynajmniej jedną ocenę (bez kolumn z ratings)
movies_with_ratings = movies.join(ratings, "movie_id", "left_semi")
print(f"Filmy z ocenami: {movies_with_ratings.count()}")
movies_with_ratings.show(5)

In [ ]:
# Join z różnymi nazwami kolumn
movies_renamed = movies.withColumnRenamed("movie_id", "id")
ratings.join(movies_renamed, ratings.movie_id == movies_renamed.id, "inner") \
    .drop("id") \
    .show(5)

### Zadanie 4
Znajdź 10 filmów z największą liczbą ocen. Użyj join z movies, żeby pokazać tytuły.

In [ ]:
# Twoje rozwiązanie:


## 8. union, distinct, dropDuplicates

In [ ]:
# union - łączenie dwóch DataFrames (muszą mieć ten sam schemat)
comedies = movies.filter(col("genres").contains("Comedy"))
dramas = movies.filter(col("genres").contains("Drama"))

comedies_or_dramas = comedies.union(dramas)
print(f"Komedie: {comedies.count()}")
print(f"Dramaty: {dramas.count()}")
print(f"Union (z duplikatami): {comedies_or_dramas.count()}")
print(f"Union (bez duplikatów): {comedies_or_dramas.distinct().count()}")

In [ ]:
# dropDuplicates - usuwanie duplikatów po wybranych kolumnach
# Np. jeden wiersz na użytkownika (pierwsza ocena)
unique_users = ratings.dropDuplicates(["user_id"])
print(f"Unikalni użytkownicy: {unique_users.count()}")

## 9. limit, sample, take

In [ ]:
# limit - zwraca nowy DataFrame z N pierwszych wierszy
ratings.limit(5).show()

# sample - losowa próbka (fraction = procent danych)
sample_ratings = ratings.sample(fraction=0.001, seed=42)
print(f"Próbka 0.1%: {sample_ratings.count()} wierszy")

# take - zwraca listę Row obiektów (do drivera!)
rows = ratings.take(3)
for r in rows:
    print(f"User {r.user_id} rated movie {r.movie_id}: {r.rating}")

## 10. cache / persist

Jeśli DataFrame jest używany wielokrotnie, warto go zcachować.

In [ ]:
from pyspark import StorageLevel

# cache() = persist(StorageLevel.MEMORY_AND_DISK)
ratings.cache()

# Pierwsza akcja - wczytuje dane do pamięci
ratings.count()

# Kolejne akcje będą szybsze
ratings.filter(col("rating") == 5.0).count()

In [ ]:
# Sprawdź w Spark UI -> Storage tab
# Zwolnij cache
ratings.unpersist()

## Zadanie końcowe

Stwórz DataFrame `user_profiles` zawierający dla każdego użytkownika:
- `user_id`
- `total_ratings` - liczba ocen
- `avg_rating` - średnia ocena
- `favorite_genre` - najczęściej oceniany gatunek (wymaga joina z movies i rozbicia genres)

Posortuj malejąco po `total_ratings` i pokaż top 20.

In [ ]:
# Twoje rozwiązanie:


In [ ]:
spark.stop()